In [1]:
#Import dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint

In [2]:
#Scrape the NASA Mars News site for latest article
nasa_mars_news_url = 'https://mars.nasa.gov/news/'
news_response = requests.get(nasa_mars_news_url)
news_soup = bs(news_response.text, 'html.parser')

#Collect the section with the latest News Title and Paragraph Text
news_stories = news_soup.find('div', class_= 'slide')

#Assign the title to variable
news_title = (news_stories.find('div', class_='content_title').text).strip('\n')

#Assign the article paragraph to variable
news_p = (news_stories.find('div', class_='rollover_description').text).strip('\n')

#Print latest news
print(f'Latest Mars News:\n{news_title}\n{news_p}')

Latest Mars News:
NASA Invites Students to Name Mars 2020 Rover
Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


In [3]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image (full size jpg) 
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

In [4]:
#Scrape JPL site for full size, feature image url
jpl_response = requests.get(jpl_url)
jpl_soup = bs(jpl_response.text, 'html.parser')

#Pull out only the background image string
current_image_url = str(jpl_soup.find('article', class_= 'carousel_item')).split("background-image: url('/")[1].split("')")[0]

#Save a complete url string for the current full sized Featured Mars Image to variable
featured_image_url = f'https://www.jpl.nasa.gov/{current_image_url}'

#Print the url for the featured image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA14254-1920x1200.jpg


In [5]:
#Visit the Mars Weather twitter account 
mars_twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_twitter_url)

In [6]:
#Scrape the latest Mars weather tweet from the page
twitter_response = requests.get(mars_twitter_url)
twitter_soup = bs(twitter_response.text, 'html.parser')

#Collect the most recent tweet
tweet = twitter_soup.find('div', class_= 'original-tweet')

#Save the tweet text for the weather report as a variable called 
mars_weather = (tweet.find('p', class_='tweet-text').text).split('pic')[0]

#Print current weather
print(f'Current Weather on Mars:\n{mars_weather}')

Current Weather on Mars:
InSight sol 361 (2019-12-02) low -98.2ºC (-144.7ºF) high -21.2ºC (-6.2ºF)
winds from the SSW at 4.9 m/s (11.0 mph) gusting to 18.9 m/s (42.3 mph)
pressure at 6.70 hPa


In [7]:
#Visit the Mars Facts webpage
mars_facts_url = 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)

In [8]:
#Scrape the Mars Space facts site
facts_response = requests.get(mars_facts_url)
facts_soup = bs(facts_response.text, 'html.parser')

#Collect the data from the table
facts_table = str(facts_soup.find('table', class_= 'tablepress'))

# print(facts_table.prettify())

#Use Pandas to convert the data to a HTML table string
facts = pd.read_html(facts_table)

#Print mars facts
facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [9]:
#Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres.
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

In [10]:
#Scrape the astrogeology site
hemisphere_response = requests.get(hemisphere_url)
hemisphere_soup = bs(hemisphere_response.text, 'html.parser')

#Collect all hemisphere titles and image urls
products = hemisphere_soup.find_all('div', class_= 'item')
hemisphere_image_urls = []

#Loop through products and create dictionary
for product in products:
    title = product.find('h3').text
    link = product.a['href']
    
    hemisphere = {
        'title': title,
        'link': f'https://astrogeology.usgs.gov{link}'
    }
    
    hemisphere_image_urls.append(hemisphere)

#Print list of dictionaries
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(hemisphere_image_urls)

[   {   'link': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced',
        'title': 'Cerberus Hemisphere Enhanced'},
    {   'link': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced',
        'title': 'Schiaparelli Hemisphere Enhanced'},
    {   'link': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced',
        'title': 'Syrtis Major Hemisphere Enhanced'},
    {   'link': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced',
        'title': 'Valles Marineris Hemisphere Enhanced'}]
